#### Game-Based Recommender (Item-Based Recommender)
Using the normalized user-by-game matrix prepared, we can work on the second variation recommender.

In [1]:
# Import Libraries
# Packages for work.
import pandas as pd
import numpy as np

In [3]:
# Reading in the normalized user by game matrix.
ubg_logged = pd.read_csv('ubg_logged.csv', index_col=0)

# Zero imputation.
ubg_logged0 = ubg_logged.fillna(0)
ubg_logged0

,Portal,Tom Clancy's Ghost Recon: Advanced Warfighter,Tom Clancy's Ghost Recon,Crysis,Crysis Warhead,Left 4 Dead 2,Torchlight,Devil May Cry 4,Batman: Arkham Asylum GOTY Edition,Battlefield: Bad Company™ 2,...,HomeWork Is Crazy / 作业疯了,Chinatris,Trine 4: The Nightmare Prince,A Way Out,Battlefield 1 ™,Kingdom Rush Vengeance,雀魂麻将(MahjongSoul),Draw & Guess,Rubber Bandits: Summer Prologue,Stacklands
76561198010430483,5.905362,3.091042,3.583519,6.719013,0.0,8.852379,6.711740,6.50279,6.659294,8.329417,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198039495811,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198040564894,0.000000,0.000000,0.000000,0.000000,0.0,7.663408,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561197994644797,6.613384,0.000000,0.000000,0.000000,0.0,10.249486,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198064970505,0.000000,0.000000,0.000000,0.000000,0.0,9.295508,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198004670799,0.000000,0.000000,0.000000,0.000000,0.0,7.781556,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561197971034129,0.000000,0.000000,0.000000,0.000000,0.0,9.446834,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561197962050254,0.000000,0.000000,0.000000,0.000000,0.0,7.070724,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198001262177,6.327937,0.000000,0.000000,0.000000,0.0,9.731453,6.602588,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
76561198084453258,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000


### Collaborative Filtering
### 2. Game-Based Collaborative Filtering

Here we need to create the game-to-game similarity matrix. <br>
We are trying to find the similarity (similarity scores) between games. <br>

The technique we are using is Cosine Similarity again. Reminder that this technique requires that the user-by-game matrix has no missing values. <br>

Implications again: <br>

The 0 values imputed this way should be interpreted with caution. <br>
In the original normalized user-by-game matrix that was created, the normalized values represents a measure of the user's preference towards the game by using playtime. A value of 0 referred to the lowest preference on the scale. <br>
However, the 0 values imputed this way indicates that the user does not own or did not play the game at all. The 0 values here should not be interpreted as a measure of user preference. <br>
Ultimately, we are using predicted preferences scores to find out how much a user would prefer a game, and then rank the games recommended. For this purpose, the magnitude of the predicted preference is not crucial. <br>

Next, we will proceed to build the game-to-game similarity matrix.

In [6]:
# Transpose the user-by-game matrix to get the game-by-user matrix. This is so that the games appear as the index, and the users appear as the columns.
gbu_logged0 = ubg_logged0.T
gbu_logged0

,76561198010430483,76561198039495811,76561198040564894,76561197994644797,76561198064970505,76561198004670799,76561197971034129,76561197962050254,76561198001262177,76561198084453258,...,76561197985630263,76561197965062542,76561197966467800,76561198024567076,76561198004803656,76561197965527053,76561197970683033,76561197991050584,76561198056613809,76561198092753159
Portal,5.905362,0.0,0.0,6.613384,0.0,0.0,0.0,0.0,6.327937,0.0,...,5.537334,0.0,0.0,6.842683,6.095825,6.673298,0.0,0.0,0.0,0.000000
Tom Clancy's Ghost Recon: Advanced Warfighter,3.091042,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,3.433987,0.0,0.0,0.0,0.000000
Tom Clancy's Ghost Recon,3.583519,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,1.386294,0.0,0.0,0.0,0.000000
Crysis,6.719013,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
Crysis Warhead,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kingdom Rush Vengeance,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.973810
雀魂麻将(MahjongSoul),0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,8.984192
Draw & Guess,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.846439
Rubber Bandits: Summer Prologue,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.931826


In [7]:
# Importing the Cosine Similarity function.
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Applying Cosine Similarity
sim_scores2 = cosine_similarity(gbu_logged0) # Applying the function to get the similarity matrix. This shows the similarity score of each game to every other game.
gtg_simscores = pd.DataFrame(sim_scores2, columns=gbu_logged0.index, index=gbu_logged0.index) # Organize the similarity matrix into a dataframe.

In [9]:
# Taking a look at the game-to-game similarity matrix.
gtg_simscores

,Portal,Tom Clancy's Ghost Recon: Advanced Warfighter,Tom Clancy's Ghost Recon,Crysis,Crysis Warhead,Left 4 Dead 2,Torchlight,Devil May Cry 4,Batman: Arkham Asylum GOTY Edition,Battlefield: Bad Company™ 2,...,HomeWork Is Crazy / 作业疯了,Chinatris,Trine 4: The Nightmare Prince,A Way Out,Battlefield 1 ™,Kingdom Rush Vengeance,雀魂麻将(MahjongSoul),Draw & Guess,Rubber Bandits: Summer Prologue,Stacklands
Portal,1.000000,0.480522,0.426845,0.318455,0.0,0.625694,0.389110,0.318455,0.540030,0.572407,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tom Clancy's Ghost Recon: Advanced Warfighter,0.480522,1.000000,0.892117,0.669019,0.0,0.314752,0.294508,0.669019,0.746928,0.733618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tom Clancy's Ghost Recon,0.426845,0.892117,1.000000,0.932645,0.0,0.274083,0.410559,0.932645,0.662454,0.647436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crysis,0.318455,0.669019,0.932645,1.000000,0.0,0.199533,0.440209,1.000000,0.493305,0.479227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crysis Warhead,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kingdom Rush Vengeance,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
雀魂麻将(MahjongSoul),0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Draw & Guess,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Rubber Bandits: Summer Prologue,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Checkpoint 4 <br>
Save the game-to-game similarity score matrix. <br>

In [10]:
gtg_simscores.to_csv("gtg_simscores.csv")

In [11]:
gtg_simscores = pd.read_csv('gtg_simscores.csv', index_col=0)

#### Item-Based Collaborative Filtering

With both the game-by-user matrix, and the game-to-game similarity score matrix, we can proceed to build the recommender function for this variation. <br>

Similar to User-Based Collaborative Filtering, we are predicting the preference (playtimes) of unplayed games for a selected user. <br>
But this time, it is based on the selected user's played games playtimes (preferences) and similarity of his unplayed to played games.

An additional condition is that the recommender must not recommend games that the selected user has played before. <br>

Several things to note: <br>
- The predicted figure is the normalized hours played, not actual hours played. <br>
- The actual hours played is not important. Just know that we are using hours played as a proxy for preference. <br>

There are 2 types of recommendations we can do: <br>
i. Based on the top played game of the selected user (most preferred game of the selected user); OR <br>
ii. Based on the aggregated preference of all the played games of the selected user <br>

For type 2i above, recommendation steps: <br>
1. Get list of played games of the selected user. <br>
2. Construct the unplayed to played games similarity score matrix. This is done by editing the original game-to-game similarity score matrix. <br>
3. Get the top 5 games with the highest similarity scores to the top played game of the selected user. <br>
No weighting is done here. Also, since we are only referencing the top played game of the selected user as preference, there is no predicting of preference here. <br>
We are purely recommending games based on their similarity to the top played game of the selected user. <br>

For type 2ii, recommendation steps: <br>

1. Get list of played games of the selected user. <br>
2. Construct the unplayed to played games similarity score matrix. This is done by editing the original game-to-game similarity score matrix. <br>
3. Convert the similarity scores of each unplayed game to each played game to weights by dividing each score by the *total* score. <br>
4. Get the selected user's playtime for all played games. <br>
5. Get the predicted playtimes (predicted preferences) of the selected user's unplayed games. Do so by multiplying the similarity weightages of each unplayed game to each played game, with the selected user's playtime for played games. <br>
6. With the list of predicted playtimes, filter out the top 5 games with the 5 highest predicted playtimes. These are the top 5 games we will recommend to the selected user. <br>

#### Type 2i - Recommending based on the top played game of the selected user. (Most preferred game of the selected user.)

In [13]:
# Step 1: Get list of played games of the selected user.
seluser_played = gbu_logged0[gbu_logged0[76561198010430483]>0].index
seluser_played = list(seluser_played)

In [16]:
# Step 2: Get the similarity scores of the unplayed games to the played games.
# First, remove the selected user's played games from the index column.
utg_simscores = gtg_simscores.drop(seluser_played)

# Then, keep only the columns of played games.
utp_simscores = utg_simscores[seluser_played]

# This gives us the unplayed-to-played similarity scores.
utp_simscores

,Portal,Tom Clancy's Ghost Recon: Advanced Warfighter,Tom Clancy's Ghost Recon,Crysis,Left 4 Dead 2,Torchlight,Devil May Cry 4,Batman: Arkham Asylum GOTY Edition,Battlefield: Bad Company™ 2,Darksiders,...,Stardew Valley,PAYDAY 2,Overcooked! 2,Deceit,Tom Clancy's Rainbow Six Siege,Dota Underlords,Prop and Seek,Team Fortress 2,Dota 2,Alien Swarm
Crysis Warhead,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000
Railroad Tycoon 3,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.115510,0.000000,0.0,0.00000,0.000000,0.0,0.115416,0.149587,0.267670
Titan Quest,0.000000,0.000000,0.000000,0.0,0.032825,0.0,0.0,0.000000,0.000000,0.0,...,0.053025,0.162558,0.000000,0.0,0.00000,0.000000,0.0,0.149761,0.178457,0.297069
Titan Quest Anniversary Edition,0.140926,0.000000,0.000000,0.0,0.124082,0.0,0.0,0.000000,0.274324,0.0,...,0.261569,0.292073,0.279625,0.0,0.19649,0.000000,0.0,0.225973,0.225487,0.355385
Total War: MEDIEVAL II - Definitive Edition,0.267401,0.552272,0.268091,0.0,0.181214,0.0,0.0,0.416791,0.412900,0.0,...,0.000000,0.077303,0.000000,0.0,0.00000,0.000000,0.0,0.234443,0.100108,0.392784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kingdom Rush Vengeance,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
雀魂麻将(MahjongSoul),0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
Draw & Guess,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
Rubber Bandits: Summer Prologue,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000


In [18]:
# Step 3: Get the top 5 games with the highest similarity scores to the top played game of the selected user.
utp_simscores[[gbu_logged0[76561198010430483].idxmax()]].sort_values(by=[gbu_logged0[76561198010430483].idxmax()], ascending=False).head(5)

,Dota 2
PUBG: BATTLEGROUNDS,0.729532
Monster Hunter: World,0.683158
Borderlands 2,0.654639
Fall Guys,0.595321
Hades,0.569934


#### Putting the above steps into a function. This is the Games-Based Recommender based on a user's most played game. (Most preferred game of the selected user.)

In [20]:
def gb_recommend(steamid,n=5):
    user_played = gbu_logged0[gbu_logged0[steamid]>0].index
    user_played = list(user_played)
    utg_simscores = gtg_simscores.drop(user_played)
    utp_simscores = utg_simscores[user_played]
    return utp_simscores[[gbu_logged0[steamid].idxmax()]].sort_values(by=[gbu_logged0[steamid].idxmax()], ascending=False).head(n)

In [21]:
gb_recommend(76561198010430483)

,Dota 2
PUBG: BATTLEGROUNDS,0.729532
Monster Hunter: World,0.683158
Borderlands 2,0.654639
Fall Guys,0.595321
Hades,0.569934


#### Applying the recommender to all users within the dataset (user-by-game matrix).

In [22]:
# Get all the user names.
users = list(ubg_logged0.index)

In [26]:
# Run the user names into the function.
gb_top5recgames = pd.DataFrame(columns=users)
for user in users:
    recommendations = gb_recommend(user)
    gb_top5recgames[user] = recommendations.index

In [27]:
gb_top5recgames

,76561198010430483,76561198039495811,76561198040564894,76561197994644797,76561198064970505,76561198004670799,76561197971034129,76561197962050254,76561198001262177,76561198084453258,...,76561197985630263,76561197965062542,76561197966467800,76561198024567076,76561198004803656,76561197965527053,76561197970683033,76561197991050584,76561198056613809,76561198092753159
0,PUBG: BATTLEGROUNDS,FINAL FANTASY VII,Overcooked,PUBG: BATTLEGROUNDS,Move or Die,Counter-Strike: Global Offensive,Plague Inc: Evolved,PAYDAY 2,Tomb Raider,Left 4 Dead 2,...,Killing Floor,Counter-Strike: Global Offensive,PAYDAY 2,Deep Rock Galactic,PAYDAY 2,PAYDAY 2,PAYDAY 2,Torchlight II,Command & Conquer™ Remastered Collection,Left 4 Dead 2
1,Monster Hunter: World,How to Survive,Dungeon Defenders II,Monster Hunter: World,Tom Clancy's Rainbow Six Siege,PUBG: BATTLEGROUNDS,Torchlight: Infinite,Borderlands 2,Left 4 Dead,Alien Swarm,...,Torchlight II,Counter-Strike: Source,Borderlands 2,Stardew Valley,Killing Floor 2,Borderlands 2,Borderlands 2,Fall Guys,FTL: Faster Than Light,Team Fortress 2
2,Borderlands 2,Freedom Force,Human: Fall Flat,Alien Swarm,Vampire: The Masquerade - Bloodhunt,PAYDAY 2,Warhammer: Vermintide 2,Warframe,Dungeon Defenders,PAYDAY 2,...,Counter-Strike: Source,Magicka,Torchlight II,Satisfactory,Left 4 Dead,Dota 2,Killing Floor,Deceit,Tropico 4,PAYDAY 2
3,Fall Guys,Attack on Titan / A.O.T. Wings of Freedom,Portal,Torchlight II,Robocraft,Dota Underlords,Resident Evil 6,Tom Clancy's Rainbow Six Siege,Poker Night at the Inventory,Fall Guys,...,Killing Floor 2,Portal 2,Portal 2,Worms Armageddon,Monster Hunter: World,Torchlight II,Counter-Strike: Global Offensive,Tom Clancy's Rainbow Six Siege,Company of Heroes 2,Monster Hunter: World
4,Hades,Alien Breed: Impact,Among Us,Fall Guys,Counter-Strike: Condition Zero,Path of Exile,Paladins,Apex Legends,Dead Island,Tom Clancy's Rainbow Six Siege,...,Orcs Must Die! 2,Portal,Portal,Kerbal Space Program,Aim Lab,Killing Floor 2,Killing Floor 2,Hades,Heroes & Generals,Borderlands 2


In [28]:
gb_top5recgames.to_csv("gb_top5recgames.csv",index=False)

#### Type ii - Recommending based on an aggregation of the similarity scores of all played games. (Aggregated preference of the selected user.)

In [29]:
# Step 1: Get list of played games of the selected user. (Same as Step 1 for 2i)
user_played = gbu_logged0[gbu_logged0[76561198010430483]>0].index
user_played = list(user_played)

In [30]:
# Step 2: Get the similarity scores of the unplayed games to the played games. (Same as Step 2 for 2i)
utg_simscores = gtg_simscores.drop(user_played)
utp_simscores = utg_simscores[user_played]
utp_simscores

,Portal,Tom Clancy's Ghost Recon: Advanced Warfighter,Tom Clancy's Ghost Recon,Crysis,Left 4 Dead 2,Torchlight,Devil May Cry 4,Batman: Arkham Asylum GOTY Edition,Battlefield: Bad Company™ 2,Darksiders,...,Stardew Valley,PAYDAY 2,Overcooked! 2,Deceit,Tom Clancy's Rainbow Six Siege,Dota Underlords,Prop and Seek,Team Fortress 2,Dota 2,Alien Swarm
Crysis Warhead,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000
Railroad Tycoon 3,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.115510,0.000000,0.0,0.00000,0.000000,0.0,0.115416,0.149587,0.267670
Titan Quest,0.000000,0.000000,0.000000,0.0,0.032825,0.0,0.0,0.000000,0.000000,0.0,...,0.053025,0.162558,0.000000,0.0,0.00000,0.000000,0.0,0.149761,0.178457,0.297069
Titan Quest Anniversary Edition,0.140926,0.000000,0.000000,0.0,0.124082,0.0,0.0,0.000000,0.274324,0.0,...,0.261569,0.292073,0.279625,0.0,0.19649,0.000000,0.0,0.225973,0.225487,0.355385
Total War: MEDIEVAL II - Definitive Edition,0.267401,0.552272,0.268091,0.0,0.181214,0.0,0.0,0.416791,0.412900,0.0,...,0.000000,0.077303,0.000000,0.0,0.00000,0.000000,0.0,0.234443,0.100108,0.392784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kingdom Rush Vengeance,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
雀魂麻将(MahjongSoul),0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
Draw & Guess,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000
Rubber Bandits: Summer Prologue,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.326290,0.0,0.31873,0.363349,0.0,0.000000,0.219164,0.000000


In [31]:
# Step 3: For each of the unplayed games, get the weighted average of the similarity scores of the played games.
utp_weights = utp_simscores.values/np.sum(utp_simscores.values)

In [32]:
# Step 4: Get the playtimes of the played games of the selected user.
user_played_playtimes = gbu_logged0[gbu_logged0[76561198010430483]>0][76561198010430483]

In [33]:
# Step 5: Predicting playtimes of the unplayed games, with an aggregating of the similarities across all played games. This is done by multiplying the weighted average of the similarity scores of the unplayed games, with the playtimes of the played games of the selected user.
unplayedgames_weighted = np.dot(utp_weights, user_played_playtimes)
unplayedgames_predictedplaytimes = pd.DataFrame(unplayedgames_weighted, index=utp_simscores.index)

In [34]:
# Step 6: Sort by predicted playtime, and recommend the top 5 unplayed games that the selected user has a high predicted playtime for.
# Recall that playtime is a proxy for preference. So here we have the predicted preferences of the selected user on unplayed games, and we are simply recommending the top 5 games with the highest predicted preferences.
recommend_top5 = unplayedgames_predictedplaytimes.sort_values(by=0, ascending=False).head(5)
recommend_top5

,0
Borderlands 2,0.016796
Counter-Strike: Source,0.015489
Killing Floor,0.014764
BioShock Infinite,0.014331
Cyberpunk 2077,0.013912


#### Putting the above steps into a function. This is the Games-Based Recommender based on an aggregation of similarity scores across all the users played games. (Aggregated preference of the selected user.)

In [35]:
def gb_recommend2(steamid,n=5):
    user_played = gbu_logged0[gbu_logged0[steamid]>0].index
    user_played = list(user_played)
    utg_simscores = gtg_simscores.drop(user_played)
    utp_simscores = utg_simscores[user_played]
    utp_weights = utp_simscores.values/np.sum(utp_simscores.values)
    user_played_playtimes = gbu_logged0[gbu_logged0[steamid]>0][steamid]
    unplayedgames_weighted = np.dot(utp_weights, user_played_playtimes)
    unplayedgames_predictedplaytimes = pd.DataFrame(unplayedgames_weighted, index=utp_simscores.index)
    recommended_topn = unplayedgames_predictedplaytimes.sort_values(by=0, ascending=False).head(n)
    return recommended_topn

In [36]:
gb_recommend2(76561198010430483)

,0
Borderlands 2,0.016796
Counter-Strike: Source,0.015489
Killing Floor,0.014764
BioShock Infinite,0.014331
Cyberpunk 2077,0.013912


#### Applying the recommender to all users within the dataset (user-by-game matrix).

In [37]:
# Get all the user names.
users = list(ubg_logged0.index)

In [38]:
# Run the user names into the function.
gb2_top5recgames = pd.DataFrame(columns=users)
for user in users:
    recommendations = gb_recommend2(user)
    gb2_top5recgames[user] = recommendations.index

In [39]:
gb2_top5recgames

,76561198010430483,76561198039495811,76561198040564894,76561197994644797,76561198064970505,76561198004670799,76561197971034129,76561197962050254,76561198001262177,76561198084453258,...,76561197985630263,76561197965062542,76561197966467800,76561198024567076,76561198004803656,76561197965527053,76561197970683033,76561197991050584,76561198056613809,76561198092753159
0,Borderlands 2,Warhammer: Vermintide 2,Alien Swarm,Alien Swarm,Alien Swarm,PAYDAY 2,Torchlight II,PAYDAY 2,Dungeon Defenders,Left 4 Dead 2,...,Counter-Strike: Source,Counter-Strike: Global Offensive,PAYDAY 2,Dota 2,PAYDAY 2,Borderlands 2,Borderlands 2,Left 4 Dead,BioShock Infinite,Team Fortress 2
1,Counter-Strike: Source,Left 4 Dead 2,Borderlands 2,Torchlight II,Portal 2,Alien Swarm,Castle Crashers,Borderlands 2,L.A. Noire,PAYDAY 2,...,Grand Theft Auto V,Magicka,Borderlands 2,Cyberpunk 2077,Left 4 Dead,Just Cause 2,Warframe,Evolve Stage 2,Cyberpunk 2077,Left 4 Dead 2
2,Killing Floor,Counter-Strike: Source,Portal 2,Counter-Strike: Source,Tom Clancy's Rainbow Six Siege,Counter-Strike: Global Offensive,Portal 2,Magicka,Trine,Alien Swarm,...,Castle Crashers,HELLDIVERS™,Torchlight II,PAYDAY 2,Killing Floor 2,Castle Crashers,PAYDAY 2,Torchlight II,Left 4 Dead 2,PAYDAY 2
3,BioShock Infinite,The Escapists 2,Portal,Overcooked! 2,Counter-Strike: Source,Cyberpunk 2077,Overcooked! 2,Killing Floor,"Warhammer 40,000: Dawn of War II",Borderlands 2,...,Torchlight II,Marvel Heroes Omega,Portal,Borderlands 2,Monster Hunter: World,Call of Duty: Black Ops,Counter-Strike: Global Offensive,Portal,Team Fortress 2,Alien Swarm
4,Cyberpunk 2077,FTL: Faster Than Light,Killing Floor 2,Killing Floor,Monster Hunter: World,Warframe,FTL: Faster Than Light,Killing Floor 2,Dead Island,Tom Clancy's Rainbow Six Siege,...,Warhammer: Vermintide 2,Apex Legends,Castle Crashers,Killing Floor,Evolve Stage 2,Stray,Borderlands: The Pre-Sequel,Magicka,Rise of the Tomb Raider,Borderlands 2


In [40]:
gb2_top5recgames.to_csv("gb2_top5recgames.csv",index=False)